# Load models from checkpoints and evaluate them on the evaluation tasks

In [94]:
import os
from pathlib import Path

import torch
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from hydra.utils import instantiate
from omegaconf import DictConfig, OmegaConf

from src.models.jump_cl import BasicJUMPModule
from src.utils import instantiate_evaluator_list

In [2]:
for i in range(1, 4):
    if not Path(f"../cpjump{i}/jump/").exists():
        print(f"Mounting cpjump{i}...")
        os.system(f"sshfs bioclust:/projects/cpjump{i}/ ../cpjump{i}")
    else:
        print(f"cpjump{i} already mounted.")

Mounting cpjump1...
Mounting cpjump2...
Mounting cpjump3...


In [66]:
ckpt_str = "../cpjump1/jump/logs/train/runs/{run}/checkpoints/epoch_{epoch:0>3}.ckpt"

run_dict = {
    "small": (run := "2023-08-03_14-37-42", "small_jump_cl", epoch := 19, ckpt_str.format(run=run, epoch=epoch)),
    "med": (run := "2023-08-07_11-55-54", "med_jump_cl", epoch := 5, ckpt_str.format(run=run, epoch=epoch)),
    "big": (run := "2023-08-01_11-37-40", "big_jump_cl", epoch := 1, ckpt_str.format(run=run, epoch=epoch)),
}

In [84]:
run, experiment, epoch, ckpt = run_dict["small"]

In [88]:
os.system(f"cat ../cpjump1/jump/logs/train/runs/{run}/.hydra/overrides.yaml")

- experiment=big_jump_cl
- trainer.devices=[0,1]


0

In [89]:
os.listdir(f"../cpjump1/jump/logs/train/runs/{run}/checkpoints")

['epoch_001.ckpt', 'last.ckpt']

## Load the config and instantiate the model, loggers and evaluators

In [5]:
initialize(version_base=None, config_path="../configs/")

hydra.initialize()

In [91]:
run, experiment, epoch, ckpt = run_dict["small"]

In [106]:
cfg = compose(
    config_name="train.yaml",
    overrides=[
        "evaluate=true",
        "eval=evaluators",
        "paths.data_root_dir=../cpjump1/",
        f"paths.output_dir=../cpjump1/jump/logs/train/runs/{run}",
        f"experiment={experiment}",
        "trainer.devices=1",
    ],
)
print(OmegaConf.to_yaml(cfg))

task_name: train
tags:
- med_jump_cl
- simple_contrastive_training
- ${model.molecule_encoder.pretrained_name}
- ${model.image_encoder.instance_model_name}
train: true
test: true
evaluate: true
compile: false
ckpt_path: null
seed: 12345
data:
  compound_transform:
    _target_: src.modules.compound_transforms.dgllife_transform.DGLPretrainedFromInchi
    add_self_loop: true
    canonical_atom_order: true
    num_virtual_nodes: 0
    explicit_hydrogens: false
  _target_: src.models.jump_cl.datamodule.BasicJUMPDataModule
  batch_size: 32
  num_workers: 24
  pin_memory: null
  prefetch_factor: 3
  collate_fn:
    _target_: src.modules.collate_fn.dgl_image.image_graph_collate_function
    _partial_: true
  transform:
    _target_: src.modules.transforms.DefaultJUMPTransform
    _convert_: object
    size: 128
    dim:
    - -2
    - -1
  force_split: false
  splitter:
    _target_: src.splitters.ScaffoldSplitter
    train: 800
    test: 200
    val: 100
    retrieval: 0
  split_path: ${path

In [60]:
ckpt

'../cpjump1/jump/logs/train/runs/2023-08-03_14-37-42/checkpoints/epoch_005.ckpt'

In [92]:
Path(ckpt).exists()

True

In [ ]:
image_encoder = instantiate(cfg.model.image_encoder)
molecule_encoder = instantiate(cfg.model.molecule_encoder)
criterion = instantiate(cfg.model.criterion)

In [99]:
model = instantiate(cfg.model)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d268fcb1-e9a5-4a33-8be2-c4196760ec65)')' thrown while requesting HEAD https://huggingface.co/timm/resnet18.a1_in1k/resolve/main/model.safetensors


Pretrained model loaded


In [100]:
model

BasicJUMPModule(
  (image_encoder): CNNEncoder(
    (backbone): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (drop_block): Identity()
          (act1): ReLU(inplace=True)
          (aa): Identity()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): ReLU(inplace=True)
        )
        (1): BasicBlock(
      

In [107]:
evaluator_list = instantiate_evaluator_list(cfg.eval, cross_modal_module=model)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
No callback configs found! Skipping..
GPU available: True (cuda), used: True
TPU availab

In [110]:
evaluator_list.run()